<a href="https://colab.research.google.com/github/michelalocatelli02/Cattolica2020/blob/main/Copy_of_Empirical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

Tue Oct 27 08:06:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 46kB/s 


In [ ]:
#Import libraries 
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten #to create the last layer
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
#re-size all the image to this 
IMAGE_SIZE=[224,224] #for the syntax of the VGG16 BUT ALSO FOR INCEPTIONV3??
train_path='/content/drive/My Drive/dataset2-master/images/TRAIN'
test_path='/content/drive/My Drive/dataset2-master/images/TEST'

In [ ]:
#import inceptionV3 library and add processing layers to the front of InceptionV3
#Here we use imagenet weights
inception=InceptionV3(input_shape=IMAGE_SIZE+[3],weights='imagenet',include_top=False)
#I do not want to use the last, I have ot thausend of categories but just 4

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
#don't train the existing layers
for layer in inception.layers:
    layer.trainable=False #trainable parameter that says that you train yout layer

In [ ]:
folders=glob('/content/drive/My Drive/dataset2-master/images/TRAIN/*')

In [ ]:
folders #to check how many classes I have 

['/content/drive/My Drive/dataset2-master/images/TRAIN/EOSINOPHIL',
 '/content/drive/My Drive/dataset2-master/images/TRAIN/LYMPHOCYTE',
 '/content/drive/My Drive/dataset2-master/images/TRAIN/NEUTROPHIL',
 '/content/drive/My Drive/dataset2-master/images/TRAIN/MONOCYTE']

In [ ]:
x=Flatten()(inception.output)# I flatten the last layer of the inception

In [ ]:
prediction=Dense(len(folders),activation='softmax')(x)#to create the last layer
#I use the softmax as activation because I have more than two categories 

In [ ]:
#create a model object 
model=Model(inputs=inception.input,outputs=prediction)

In [ ]:
#view the structure of the model
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [ ]:
#tell the model what cost and optimization method to use 
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']

)

In [ ]:
#Use the Image Data Generator to import the images from the dataset 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
#import data 
training_set=train_datagen.flow_from_directory('/content/drive/My Drive/dataset2-master/images/TRAIN',
                                               target_size=(224,224),
                                               batch_size=16,
                                               class_mode='categorical')

Found 9976 images belonging to 4 classes.


In [ ]:
test_set=test_datagen.flow_from_directory('/content/drive/My Drive/dataset2-master/images/TEST',
                                          target_size=(224,224),
                                          batch_size=16,
                                          class_mode='categorical')

Found 2515 images belonging to 4 classes.


In [ ]:
#fit the model
r=model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=10,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)

NameError: ignored

In [ ]:
 import matplotlib.pyplot as plt

In [ ]:
#plot the loss
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'],label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

NameError: ignored

In [ ]:
#plot the accuracy

In [ ]:
#plot the confusion matrix 